Please refer to the Reinforcement Learning Jupyter notebook in course materials.
Answer questions 1-3 below considering any Nim game reinforcement learning model.

1. [10 pts] Describe the environment in the Nim learning model.

#### Ans.

In a Nim game reinforcement learning model, the environment describes the game's setup, rules, and dynamics. It starts with several heaps/piles containing objects (number of heaps and items within each heap can differ), and the game progresses through players taking turns to remove objects from these heaps. Actions in this environment consist of choosing a heap and deciding how many objects to remove. The state transitions according to these actions, updating the number of objects left in each heap. The game concludes when all heaps are empty. In Nim, the environment provides feedback through rewards (typically binary) at the game's end, based on winning or losing.The exact reward structure can vary and significantly affects the learning process. It also enforces the Nim game rules, including the turn-based play (agents take turns consistently and only the active agent can take actions) and legal moves.

2. [10 pts] Describe the agent(s) in the Nim learning model (Hint, not just the Q-learner). Is Guru an agent?

#### Ans.

In the Nim game learning model, agents are the entities making decisions about which moves to make. There can be multiple agents involved, depending on the model setup:

- Q-Learner: This agent uses Q-learning, a model-free reinforcement learning algorithm to learn the value of taking a given action (i.e., how many objects to remove from which pile) in a given state. It maintains a Q-table (or Q-values) that represent the expected utility (or "quality") of taking an action in a particular state and updates this table as it learns from experience, , aiming to maximize cumulative rewards.

- Random Agents: These agents make decisions randomly and are often used as baseline competitors to evaluate the performance of more sophisticated agents like Q-learners. 

- Heuristic-based Agents: These agents follow specific strategies or heuristics predefined by domain knowledge about the game. They might not adapt through learning as Q-learners do but can serve as a benchmark or training partner for learning agents. The heuristic simplifies the learning problem by prioritizing immediate rewards, but must maintain a delicate balance to prevent oversimplification, ensuring that the agent still considers significant long-term consequences [1].

Guru can refer to an expert or highly skilled player, could be incorporated as an agent with a predefined strategy or expert moves. It serves various roles such as providing guidance to the learning agent, acting as an opponent, or being used for teaching scenarios where the learning agent tries to imitate or learn from the Guru's moves. Whether or not the Guru is considered an agent in the learning model depends on its role; if it makes decisions and interacts with the environment, it can be considered an agent.

3. [10 pts] Describe the reward and penalty in the Nim learning model.

#### Ans.

- Reward: In the Nim game, a reward is typically given for a move that leads the agent closer to winning the game. The exact nature of the reward can vary; for example, it might be a positive numerical value assigned for moves that reduce the number of piles or the total number of objects in a way that is advantageous according to the game's winning strategy.

- Penalty: Similarly, a penalty (or negative reward) is applied for moves that lead the agent away from winning or violate the game's rules. For instance, a penalty might be given for making a move that allows the opponent to be in a winning position on their next turn.

4. [10 pts] How many possible states there could be in the Nim game with a maximum of 10
items per pile and 3 piles total? (This problem requires a number for its answer, not merely
a closed-form expression.)

#### Ans. 

Since each pile can have between 0 and 10 items (inclusive), there are 11 possible states for each pile (0 through 10 items). Given 3 piles, the total number of states can be calculated by considering all the combinations of items across these piles. For each pile having 11 possible states, the total number of combinations for 3 piles would be 
$ 11^3 $.

In [25]:
# Calculating the total number of possible states in the Nim game
possible_states_per_pile = 11  # 0 to 10 items, inclusive
total_piles = 3
total_possible_states = possible_states_per_pile ** total_piles
total_possible_states

1331

5. [10 pts] How many possible unique actions are there for player 1 to take as their first action
in a Nim game with 10 items per pile and 3 piles total? (This problem also requires a
number for its answer, not merely a closed-form expression.)

#### Ans. 

In a Nim game with 3 piles, each containing 10 items, the total number of unique actions available for player 1's first move is 30. This is because for each of the 3 piles, the player can choose to remove any number of items from 1 to 10, resulting in 10 possible actions per pile. Multiplying the 10 actions by the 3 piles gives us the total unique actions Player 1 can take for their initial move.

In [26]:
# Calculating the total number of unique actions for the first move
actions_per_pile = 10  # 1 to 10 items can be removed
total_piles = 3
total_unique_actions_first_move = actions_per_pile * total_piles
total_unique_actions_first_move

30

6. [10 pts] Do you think a Q-learner can beat the Guru player? Why or why not? Be thorough.

#### Ans. 

In theory, a Q-learner has the potential to learn the perfect strategy for Nim and thus be capable of beating or at least matching the Guru player under ideal conditions. However, this outcome depends heavily on the Q-learner's configuration, the learning environment, and the amount of training. The main challenge is that any mistake against the Guru leads to a loss, limiting the Q-learner's ability to explore without losing. This makes it hard for the Q-learner to discover and learn the optimal strategy. Q-learning faces challenges like slow learning and handling large state spaces, which could hinder the Q-learner's progress. While Nim's simple rules help, the need for perfect play to beat the Guru means these challenges are significant and could prevent the Q-learner from reaching its full potential.

7. [40 pts] Find a way to improve the provided Nim game learning model. (Hint: How about
penalizing the losses? Hint: It is indeed possible to find a better solution, which improves
the way Q-learning updates its Q-table). You must code a solution and also demonstrate
the improvement by reporting its performance against players (Random, Guru).
Do not put the Guru player’s operating code inside the learning module, as this would
defeat the purpose of reinforcement learning. However, you may train your improved Q-
learner by having it playing against a Guru; using those games as experience is legitimate
reinforcement learning.

#### Ans.

In [27]:
import numpy as np
from random import randint, choice
from collections import defaultdict

# The number of piles is 3

# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game()->list:
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(_st:list)->(int,int):
    xored = _st[0] ^ _st[1] ^ _st[2]
    if xored == 0:
        return nim_random(_st)
    for pile in range(3):
        s = _st[pile] ^ xored
        if s <= _st[pile]:
            return _st[pile]-s, pile

# Random Nim player
def nim_random(_st:list)->(int,int):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

# Q-learner Nim player
def nim_qlearner(_st:list)->(int,int):
    global qtable
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    return move, pile  # action

def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile * ITEMS_MX + move - 1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)


Based on the hints, few modifications are made to the qlearn code:
- Penalizing Losses: Penalize the Q-learner for making losing moves by assigning a negative reward  will be factored into the Q-table updates using `loss_penalty`.

-  Epsilon-Greedy Strategy for Exploration and Exploitation: Using an epsilon-greedy strategy allows the Q-learner to both explore new actions with a probability of epsilon and exploit the best-known action with a probability of 1 - epsilon. Over time, the decay of epsilon reduces exploration, placing more emphasis on exploitation as the Q-learner becomes more experienced.

- Learning Rate Decay: Decaying the learning rate (Alpha) by `alpha_decay` over time allows the Q-learner to make larger updates to the Q-table initially when it knows less and smaller updates as it becomes more knowledgable.

-  Training with Guru opponent: The training function includes games against the Guru, alternating between playing as the starting player and the second player based on random choice.

In [28]:
def nim_qlearn_enhanced(_n, epsilon_decay, alpha_decay, loss_penalty, guru_games_ratio=0.5):
    global qtable, Alpha, epsilon
    qtable = np.zeros((ITEMS_MX + 1, ITEMS_MX + 1, ITEMS_MX + 1, ITEMS_MX * 3), dtype=np.float32)
    Alpha, epsilon = 1.0, 1.0  # Initialize Alpha and epsilon

    for _ in range(_n):
        st1 = init_game()
        if np.random.rand() < guru_games_ratio:  # Decide whether to play against the Guru
            opponent = nim_guru
        else:
            opponent = nim_random

        while True:
            if np.random.rand() < epsilon:  # Epsilon-greedy strategy
                move, pile = nim_random(st1)  # Exploration
            else:
                move, pile = nim_qlearner(st1)  # Exploitation

            st2 = list(st1)
            st2[pile] -= move

            if st2 == [0, 0, 0]:  # Q-learner wins
                qtable_update(Reward, st1, move, pile, 0)
                break
            elif any(x < 0 for x in st2):  # Q-learner makes an invalid move
                qtable_update(-loss_penalty, st1, move, pile, 0)
                break

            # Guru's turn
            move, pile = opponent(st2)
            st2[pile] -= move
            if st2 == [0, 0, 0]:  # Guru wins
                qtable_update(-loss_penalty, st1, move, pile, 0)
                break

            qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            st1 = st2

        Alpha *= alpha_decay  # Decay learning rate
        epsilon *= epsilon_decay  # Decay epsilon

In [29]:
def game(_a, _b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[_a] if side == 'A' else Engines[_b]
        move, pile = engine(state)
        state[pile] -= move
        if state == [0, 0, 0]:
            return side
        side = 'B' if side == 'A' else 'A'

def play_games(_n, _a, _b):
    wins = defaultdict(int)
    for _ in range(_n):
        wins[game(_a, _b)] += 1
    print(f"{_n} games, {_a:>8s}{wins['A']:5d}  {_b:>8s}{wins['B']:5d}")
    return wins['A'], wins['B']

In [38]:
%%time
# Q-learning parameters
Gamma, Reward = 0.8, 100.0

# Enhanced Q-learner training
nim_qlearn_enhanced(10000, epsilon_decay = 0.995, alpha_decay= 0.995, loss_penalty = 70)  

CPU times: user 531 ms, sys: 36.2 ms, total: 567 ms
Wall time: 557 ms


In [41]:
%%time

# Engines
Engines = {'Random': nim_random, 'Guru': nim_guru, 'Qlearner': nim_qlearner}

# Play games to test the enhanced Q-learner
play_games(1000, 'Random', 'Random') ;
play_games(1000, 'Qlearner', 'Random')
play_games(1000, 'Qlearner', 'Guru')

1000 games,   Random  519    Random  481
1000 games, Qlearner  660    Random  340
1000 games, Qlearner   14      Guru  986
CPU times: user 56.4 ms, sys: 5.62 ms, total: 62.1 ms
Wall time: 64.4 ms


(14, 986)

### References:

1. Cheng, C. A., Kolobov, A., & Swaminathan, A. (2021). Heuristic-Guided Reinforcement Learning. arXiv preprint arXiv:2106.02757v2. 
2. Stanford University. (n.d.). MDPs: epsilon-greedy. Retrieved from https://stanford-cs221.github.io/autumn2020-extra/modules/mdps/epsilon-greedy.pdf